# Import libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [2]:
r = requests.get('https://www.jamesallen.com/loose-diamonds/all-diamonds/')
r.content

b'\r\n\r\n<!DOCTYPE html>\r\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en-us">\r\n<head id="Head1">\r\n<title>All Diamonds - Magnified in 360\xc2\xb0 HD | JamesAllen.com</title>\r\n <style type="text/css" href="/mini/main.min.css" rel="stylesheet">/*!! ver.1.1 GeneralElements.css */\r\nbody{color:#1b1b1b;font-family:Nunito Sans;font-size:14px;font-weight:normal}body.DisableScroll{overflow:hidden}body.InnerTitle{margin:0}wbr:after{content:"\xe2\x80\x8b"}#body1 .ui-loader{display:none}#BodyContainer.PagingHeader #Pane{margin-top:0;border:none;min-height:0}#BodyContainer.PagingHeader header,#BodyContainer.PagingHeader .DiamondExpert,#BodyContainer.PagingHeader #ResultsTabs,#BodyContainer.PagingHeader .SearchSepLine,#BodyContainer.PagingHeader .ResultsCount,#BodyContainer.PagingHeader .SortBy,#BodyContainer.PagingHeader #BreadCrumbsPane,#BodyContainer.PagingHeader #Filters,#BodyContainer.PagingHeader #Gem>*:not(#SearchResults),#BodyContainer.PagingHeader #SearchPane>*:not(#SearchRes

In [165]:
# Creating an instance of BeautifulSoup
soup = bs(r.content, "html5lib")

listing_divs = soup.find('div', attrs={'id':'data-page-container'})
print(listing_divs.prettify())

<div class="gallery-grid--qTRRj" id="data-page-container" style="margin-top:16px">
 <div class="" data-page="1" data-qa="galleryItem_1_0" id="_1_0">
  <div class="promotion-container--LUQg6">
   <div>
    <div class="super-zoom-container--vgW0F" data-qa="superZoom">
     <span class="icon-search2 super-zoom--fj6MM" title="Super Zoom">
     </span>
    </div>
   </div>
   <div class="item-container--n_Uh3" style="overflow:hidden">
    <a class="image-container--XXHhO" data-ajax-mode="disabled" data-container="#WidePane" href="loose-diamonds/round-cut/1.00-carat-h-color-si1-clarity-excellent-cut-sku-13729999" title=" 1.00 CARAT H SI1 EXCELLENT CUT ROUND DIAMOND">
     <div>
      <div>
       <div class="g-outer--1Wgob">
        <div class="loader-container--3BOqs">
         <div class="loader--18qrK">
         </div>
        </div>
        <div data-qa="gallery_diamond__1_0_player" style="line-height:0;padding-bottom:79.20792079207921%;height:0">
         <div class="imgBgContainer--14v

In [166]:
first = listing_divs.select('div[id*=_1_0]')[0]
first

<div class="" data-page="1" data-qa="galleryItem_1_0" id="_1_0"><div class="promotion-container--LUQg6"><div><div class="super-zoom-container--vgW0F" data-qa="superZoom"><span class="icon-search2 super-zoom--fj6MM" title="Super Zoom"></span></div></div><div class="item-container--n_Uh3" style="overflow:hidden"><a class="image-container--XXHhO" data-ajax-mode="disabled" data-container="#WidePane" href="loose-diamonds/round-cut/1.00-carat-h-color-si1-clarity-excellent-cut-sku-13729999" title=" 1.00 CARAT H SI1 EXCELLENT CUT ROUND DIAMOND"><div><div><div class="g-outer--1Wgob"><div class="loader-container--3BOqs"><div class="loader--18qrK"></div></div><div data-qa="gallery_diamond__1_0_player" style="line-height:0;padding-bottom:79.20792079207921%;height:0"><div class="imgBgContainer--14vJZ"><img alt=" 1.00 Carat H SI1 Excellent Cut round diamond" src="https://ion.r2net.com/sgmdirect/photoID/21600168/Diamond/13729999/Diamond-round-1-Carat_3_first_.jpg" style="width:100%"/></div><canvas cl

In [133]:
dia_url = first.select('a[class*=image-container]')[0]['href']
dia_url = 'https://www.jamesallen.com/'+dia_url
print(dia_url)

https://www.jamesallen.com/loose-diamonds/round-cut/1.00-carat-h-color-si1-clarity-excellent-cut-sku-14125269


In [134]:
data = first.select('a[class*=item-description]')[0].select('span')[0].string
data = data.split()
carat = data[0]
color = data[2]
clarity = data[3]
cut = data[4]
shape = data[6]
print(carat)
print(color)
print(clarity)
print(cut)
print(shape)
price = first.select('div[class*=salePrice]')[0].string
print(price)

1.00
H
SI1
Excellent
round
$5,550


In [190]:
# Creating a function to retrieve data from a list of <div>s

def retrieve_data(page, listing_divs):    
    listing_list = []
    for index in range(23):
        each_listing = []
        current_diamond = listing_divs.select('div[id*=_{page}_{index}]'.format(page=page, index=index))[0]
        
        url = current_diamond.select('a[class*=image-container]')[0]['href']
        url = 'https://www.jamesallen.com/'+url
  
        data = current_diamond.select('a[class*=item-description]')[0].select('span')[0].string
        data = data.split()
        carat = data[0]
        color = data[2]
        clarity = data[3]
        cut = data[4]
        shape = data[6]
        price = current_diamond.select('div[class*=salePrice]')[0].string


        each_listing.append(url)
        each_listing.append(carat)
        each_listing.append(color)
        each_listing.append(clarity)
        each_listing.append(cut)
        each_listing.append(shape)
        each_listing.append(price)

        listing_list.append(each_listing)

    return listing_list

In [194]:
# getting data from the first page
r = requests.get('https://www.jamesallen.com/loose-diamonds/all-diamonds/')
    
# Getting a BeautifulSoup instance to be able to retrieve data
soup = bs(r.content, "html5lib")

listing_divs = soup.find('div', attrs={'id':'data-page-container'})
    
all_pages_parsed = []
    
all_pages_parsed.extend(retrieve_data(1, listing_divs))
    

In [195]:
# Looping and getting data from the next 10 pages

url_prefix = 'https://www.jamesallen.com/loose-diamonds/all-diamonds/page-'
page_number = 2

pages = 10

for url in range(pages):
    target_page = url_prefix + str(page_number) + '/'
    page_number += 1

    r = requests.get(target_page)
    
    # Getting a BeautifulSoup instance to be able to retrieve data
    soup = bs(r.content, "html5lib")
    
    listing_divs = soup.find('div', attrs={'id':'data-page-container'})

    one_page_parsed = retrieve_data(page_number-1, listing_divs)
    all_pages_parsed.extend(one_page_parsed)

In [197]:
df = pd.DataFrame(all_pages_parsed, columns=['url','carat','color','clarity','cut','shape','price'])
df.to_csv('Diamonds')